<a href="https://colab.research.google.com/github/kgudalewski/Stocks_analysis/blob/main/AAPL_yt_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
from datetime import datetime, timedelta
import time
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
import matplotlib.pyplot as plt
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
!pip install emoji==0.6.0
import emoji

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
df = pd.read_csv('/content/drive/MyDrive/titles.csv',encoding='utf-16',index_col=0)
df.head()

,Search_fraze,title
0,apple stock prices,🔴Live Tesla & Apple Signals | Free 5m Chart TS...
1,apple stock prices,Apple Stock Price Trend Analysis Financial Mar...
2,apple stock prices,Apple Stock price📈 after iphone 15 launch #sho...
3,apple stock prices,APPL Stock Valuation (APPLE STOCK) | Calculati...
4,apple stock prices,Apple and Netflix Stock Prices Surge: Analysts...


In [23]:
sentiment = pipeline("sentiment-analysis", model="finiteautomata/bertweet-base-sentiment-analysis")

# tokenizer = AutoTokenizer.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis")
# model = AutoModelForSequenceClassification.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis")

Device set to use cpu


In [24]:
for i in range(30):
  print(f"{i} ------------------")
  print(df.iloc[i]["title"])
  print(sentiment.predict(df.iloc[i]["title"]))

# will be better to use classification model, with classes -1, 0, 1 for negative, neutral, positive info respectively
# there will be need for handy labeling data

0 ------------------
🔴Live Tesla & Apple Signals | Free 5m Chart TSLA AAPL Stock Trading Analysis & Prediction
[{'label': 'NEU', 'score': 0.9648159742355347}]
1 ------------------
Apple Stock Price Trend Analysis Financial Market 0205
[{'label': 'NEU', 'score': 0.957148551940918}]
2 ------------------
Apple Stock price📈 after iphone 15 launch #shorts #apple #iphone
[{'label': 'NEU', 'score': 0.9227747917175293}]
3 ------------------
APPL Stock Valuation (APPLE STOCK) | Calculating Intrinsic Value
[{'label': 'NEU', 'score': 0.8779382705688477}]
4 ------------------
Apple and Netflix Stock Prices Surge: Analysts Boost Targets Amid Strong Earnings Reports
[{'label': 'POS', 'score': 0.9322102665901184}]
5 ------------------
Best Stocks to Buy: Apple Stock vs. Microsoft Stock | MSFT Stock vs. AAPL Stock
[{'label': 'POS', 'score': 0.518457293510437}]
6 ------------------
Aapl stock analysis today: Apple stock will fly to the moon?
[{'label': 'NEU', 'score': 0.8094589114189148}]
7 -----------

let's summarize concatenated text

In [25]:
pipe = pipeline("summarization", model="facebook/bart-large-cnn")

Device set to use cpu


In [31]:
titles_text = " ".join(list(df.title))
summarization_list = []
for i in range(len(titles_text)%100):
  summarization_list.append(pipe(titles_text[i*100:(i+1)*100],min_length=10, max_length=20)[0]["summary_text"])

summarization_list

['Live Tesla & Apple Signals | Free 5m Chart TSLA AAPL Stock',
 'Apple Stock price will rise after the launch of the new iPhone 15.',
 'Apple and Netflix Stock P. e #iphone APPL Stock Valuation (AP',
 'Rices surge as Analysts Boost Targets Amid Strong Earnings Reports. Best St',
 'AAPL stock analysis today: Apple stock will fly to the top. Apple stock',
 'Apple stock is on the rise. Is Apple on its way to the moon',
 'Apple is one of the best stocks to invest in now. The company is expected to',
 'AAPL MICROSOFT Vs. APPLE 2021-2026',
 'Apple Stock Is up BIG 4.5% in the past week. Apple stock is',
 'Apple Stock Analysis for Wednesdays. AAPL Price Predictions - Apple Stock Analysis',
 'APPLE stock analysis - ($AAPL) Stock price prediction - is it time',
 'Apple Stock Tumbles -2% As FOMC Decision Looms. Apple Stock',
 'Apple Stock Analysis for Wednesday, J. Trend Analysis Financial Market 0218 AAPL Price',
 'Apple stock is up 2.7% this week. The stock is down 1.',
 ' NVIDIA Vs. APPLE 2021

In [32]:
# sentiment on summarization
for i, summary in enumerate(summarization_list):
  print(f"{i} ------------------")
  print(summary)
  print(sentiment.predict(summary))


0 ------------------
Live Tesla & Apple Signals | Free 5m Chart TSLA AAPL Stock
[{'label': 'NEU', 'score': 0.9436664581298828}]
1 ------------------
Apple Stock price will rise after the launch of the new iPhone 15.
[{'label': 'POS', 'score': 0.8023853302001953}]
2 ------------------
Apple and Netflix Stock P. e #iphone APPL Stock Valuation (AP
[{'label': 'NEU', 'score': 0.9058467149734497}]
3 ------------------
Rices surge as Analysts Boost Targets Amid Strong Earnings Reports. Best St
[{'label': 'POS', 'score': 0.8984957337379456}]
4 ------------------
AAPL stock analysis today: Apple stock will fly to the top. Apple stock
[{'label': 'POS', 'score': 0.9271851181983948}]
5 ------------------
Apple stock is on the rise. Is Apple on its way to the moon
[{'label': 'POS', 'score': 0.9460065364837646}]
6 ------------------
Apple is one of the best stocks to invest in now. The company is expected to
[{'label': 'POS', 'score': 0.9893390536308289}]
7 ------------------
AAPL MICROSOFT Vs. APPL

In [33]:
for summary in summarization_list:
  result = 0
  if sentiment.predict(summary)[0]["label"] == "NEU":
    result = result - 1 * (sentiment.predict(summary)[0]["score"])
  elif sentiment.predict(summary)[0]["label"] == "POS":
    result = result + 1 * (sentiment.predict(summary)[0]["score"])

result = result/len(summarization_list)
result

# the result is average of pos/neg prediction multiplied by score
# it says whether general sentiment taken from youtube videos' titles on particular topic are positive or negative

-0.010201914059488396